# Five Days Plan 

<b>Goals:</b>
1. introduce Julia as instrument for instrument to solve problems of TNO The Hague. Matlab to be phased out. TNO further stranger for students;  
2. reduce wall clock time for assembly by replacing a interpreted double loop (destination and source triangles) over all elements in the mesh from interpreted version by a compiled version; 

## Day 1: Interview - Refine Plan Below 

1. details function for first (inner) integral;
2. agree on test data and test results for function for inner integral;
3. agree on quadragture rules for prisms. Use product of quadrature on triangle times quadrature 
4. details assembly matrix and vector for reference element. Agree on test data and test reference results;
5. details assembly matrix and vector for physical element. Agree on test data and test reference results;
6. details triangular meshes to be used for assembly. Pass Matlab meshes to Julia? Agree on test data and test reference results;
7. details assembly matrix and vector for entire mesh. Agree on test data and test reference results;

## Day 2: Function for Analytical Expression of First (Inner) Integral 

Include testing. 

## Day 3: Matrix and Vector of Single Triangular Element 

Include testing. 

## Day 4: Matrix and Vector Assembly - Linear Solve - Post-Processing 

Include testing. 

## Day 5: Buffer 